In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
# EDA -> Utils
# Pipeline -> Settings
# Model -> Tuning
# Presentation / Read Me

In [3]:
# load images with labels
# train & validate splits

# load test data from other files (~1500 images)

In [4]:
images_dir = 'data/HAM10000_images/'
df = pd.read_csv('data/HAM10000_metadata')
df.head(5)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [5]:
#identify class imbalance
# df.groupby('dx').size() ,   df.groupby('dx').size()/df.shape[0]

In [33]:
classes = df['dx'].unique().tolist()
classes

['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']

In [6]:
df = df.drop_duplicates(subset=['lesion_id'])

In [34]:
paths = df['image_id'].apply(lambda x: x + '.jpg')
paths = paths.tolist()
labels = df['dx'].apply(lambda x: classes.index(x))
labels

0        0
2        0
4        0
6        0
8        0
        ..
9994     6
10001    6
10004    6
10008    6
10010    6
Name: dx, Length: 7470, dtype: int64

In [40]:
test = list(zip(paths,labels))
test[5]

('ISIC_0025276.jpg', 0)

In [9]:
df.head(10)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face,vidir_modern
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back,vidir_modern
10,HAM_0001396,ISIC_0025276,bkl,histo,55.0,female,trunk,vidir_modern
11,HAM_0004234,ISIC_0029396,bkl,histo,85.0,female,chest,vidir_modern
13,HAM_0001949,ISIC_0025767,bkl,histo,70.0,male,trunk,vidir_modern
15,HAM_0007207,ISIC_0031326,bkl,histo,65.0,male,back,vidir_modern
16,HAM_0001601,ISIC_0025915,bkl,histo,75.0,male,upper extremity,vidir_modern


In [41]:
img = mpimg.imread(images_dir+paths[7])
# imgplot = plt.imshow(img)
# plt.show()

In [42]:
img.shape

(450, 600, 3)

In [36]:
ds_train = tf.data.Dataset.from_tensor_slices((paths,labels))
def read_image(file,label):
    img = tf.io.read_file(images_dir+file)
    img = tf.image.decode_image(img,channels=3,dtype=tf.float32)
    return img,label
def augment(image,label):
    #stuff?
    #image = tf.image.reshape(image,)
    return image,label

ds_train = ds_train.map(read_image).map(augment).batch(2)

In [37]:
ds = tf.data.Dataset.from_tensor_slices((paths,labels))
ds = ds.map(read_image).map(augment).batch(2)

In [26]:
ds.take(1)

<TakeDataset shapes: (<unknown>, (None,)), types: (tf.float32, tf.string)>

In [43]:
cnn = keras.Sequential([
    #cnn
    layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(450,600,3)),
    layers.MaxPooling2D((2,2)),
    #dense
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(10,activation='softmax')   
])

cnn.compile(optimizer='adam',
           loss='sparse_categorical_crossentropy',
           metrics=['accuracy'])


In [44]:
cnn.fit(ds,epochs=1)

3735/3735 [==============================] - 6135s 2s/step - loss: 8.6984 - accuracy: 0.9711


In [45]:
cnn.save('./my_model')

INFO:tensorflow:Assets written to: ./my_model/assets
